# <div align = "center" style = "color:rgb(50, 250, 250);"> Lat Default Outstanding Principal </div>

## Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')


from datetime import datetime

# %% [markdown]
## Configure Settings
# Set options or configurations as needed

# Example: pd.set_option('display.max_columns', None)


import os
from nbclient import NotebookClient
from nbformat import read
import time


### Documentation of SQL Query Logic

This query retrieves outstanding principal amounts for loan accounts at specific default milestones (10/30 days after due dates) where the account has default flags (`defFPD10`, `defFPD30`, `defSPD30`, or `defTPD30`) set to `1`. The logic is broken down into multiple CTEs for clarity.

---

### **1. `all_default_account` (Base CTE)**
**Purpose**: Identify all loan accounts with at least one active default flag.  
**Source Table**:  
- `risk_credit_mis.loan_master_table`  
**Columns Selected**:  
- `digitalLoanAccountId`, `loanAccountNumber`, `firstDueDate`, `secondDueDate`, `thirdDueDate`  
- `defFPD30`, `defSPD30`, `defTPD30` (default flags for 30-day milestones)  
**Filter Condition**:  
```sql 
WHERE defFPD10=1 OR defFPD30=1 OR defSPD30=1 OR defTPD30=1
```
**Output**: All loans with at least one default flag active.

---

### **2. Default Principal Outstanding CTEs**
**Purpose**: For each default type, capture the **PRINCIPALARREARS** from `core_raw.loan_accounts` at specific dates relative to due dates. Uses a 1-day buffer to handle non-business days.  

#### **a. `defpd10_pricipal_outstanding` (First Payment Default @ 10 Days)**
- **Applicable Loans**: `defFPD10=1`  
- **Lookup Date**: `firstDueDate + 10/11 days`  
- **Logic**:  
  ```sql
  JOIN core_raw.loan_accounts core 
    ON core.ACCOUNTNUMBER = loan.loanAccountNumber 
    AND (core._Partitiondate = DATE_ADD(firstDueDate, INTERVAL 10 DAY)
         OR core._Partitiondate = DATE_ADD(firstDueDate, INTERVAL 11 DAY))
  WHERE defFPD10=1
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY loanAccountNumber 
    ORDER BY PRINCIPALARREARS DESC
  ) = 1
  ```
- **Result**: Highest `PRINCIPALARREARS` for each loan on day 10/11 after the first due date.

#### **b. `defpd30_pricipal_outstanding` (First Payment Default @ 30 Days)**
- **Applicable Loans**: `defFPD30=1`  
- **Lookup Date**: `firstDueDate + 30/31 days`  
- **Same logic as above**, but for 30-day default.

#### **c. `defspd30_pricipal_outstanding` (Second Payment Default @ 30 Days)**
- **Applicable Loans**: `defSPD30=1`  
- **Lookup Date**: `secondDueDate + 30/31 days`  
- **Uses `secondDueDate`** instead of `firstDueDate`.

#### **d. `deftpd30_pricipal_outstanding` (Third Payment Default @ 30 Days)**
- **Applicable Loans**: `defTPD30=1`  
- **Lookup Date**: `thirdDueDate + 30/31 days`  
- **Uses `thirdDueDate`**.

---

### **3. `default_outstanding_principal` (Combined Results)**
**Purpose**: Consolidate results from all default CTEs into a single row per loan account.  
**Logic**:  
```sql
SELECT 
  allaccount.digitalLoanAccountId,
  allaccount.loanAccountNumber,
  day10dpd.defpd10_outstanding_principal,  -- From defpd10 CTE
  firstdpd.defpd30_outstanding_principal,   -- From defpd30 CTE
  seconddpd.defspd30_outstanding_principal, -- From defspd30 CTE
  thirddpd.deftpd30_outstanding_principal,  -- From deftpd30 CTE
  allaccount.firstDueDate,
  allaccount.secondDueDate,
  allaccount.thirdDueDate,
  allaccount.defFPD30 AS loan_defFPD30,     -- Original flags
  allaccount.defSPD30 AS loan_defSPD30,
  allaccount.defTPD30 AS loan_defTPD30
FROM all_default_account allaccount
LEFT JOIN defpd10_pricipal_outstanding day10dpd 
  ON allaccount.loanAccountNumber = day10dpd.loanAccountNumber
LEFT JOIN defpd30_pricipal_outstanding firstdpd 
  ON allaccount.loanAccountNumber = firstdpd.loanAccountNumber
LEFT JOIN defspd30_pricipal_outstanding seconddpd 
  ON allaccount.loanAccountNumber = seconddpd.loanAccountNumber
LEFT JOIN deftpd30_pricipal_outstanding thirddpd 
  ON allaccount.loanAccountNumber = thirddpd.loanAccountNumber
```
- **Left Joins**: Ensures all loans from `all_default_account` are included, even if they don’t have data in a specific default CTE.  
- **Output**: One row per loan with columns for each default type’s principal arrears (null if not applicable).

---

### **4. Final `SELECT`**
```sql
SELECT * FROM default_outstanding_principal;
```
**Output**: All columns from the `default_outstanding_principal` CTE.

---

### **Key Notes**
1. **Date Handling**:  
   - Uses `DATE_ADD` and a 1-day buffer (e.g., `+10/11 days`) to handle weekends/holidays where data might not be captured.
   
2. **Deduplication**:  
   - `QUALIFY ROW_NUMBER() ... =1` ensures only one record per loan is kept, prioritizing the **highest principal arrears** if multiple dates match.

3. **Default Flags**:  
   - The base CTE (`all_default_account`) filters loans with **any** default flag active.
   - Individual CTEs further filter loans relevant to their specific default type.

4. **Null Values**:  
   - If a loan has `defFPD10=1` but no matching `core_raw.loan_accounts` record on day 10/11, `defpd10_outstanding_principal` will be `null`.

---

### **Summary Flow**
1. Identify defaulted loans →  
2. For each default type, fetch principal arrears at defined days post-due date →  
3. Combine results into a unified structure →  
4. Output all data.

## Query

In [2]:
query = """
create or replace table dap_ds_poweruser_playground.lat_default_outstanding_principal as 
with all_default_account as (select digitalLoanAccountId,loanAccountNumber,firstDueDate,secondDueDate,thirdDueDate,defFPD30,defSPD30,defTPD30 from 
`risk_credit_mis.loan_master_table`
where defFPD10=1 or defFPD30=1 or defSPD30=1 or defTPD30=1)
,
defpd10_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd10_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 10 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 11 DAY)=core._Partitiondate)
where defFPD10=1 qualify row_number() over (partition by loanAccountNumber order by defpd10_outstanding_principal desc )=1),
defpd30_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defFPD30=1 qualify row_number() over (partition by loanAccountNumber order by defpd30_outstanding_principal desc )=1),
defspd30_pricipal_outstanding as (
select loanAccountNumber,secondDueDate,PRINCIPALARREARS defspd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(secondDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(secondDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defSPD30=1 qualify row_number() over (partition by loanAccountNumber order by defspd30_outstanding_principal desc )=1),
deftpd30_pricipal_outstanding as (
select loanAccountNumber,thirdDueDate,PRINCIPALARREARS deftpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(thirdDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(thirdDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defTPD30=1 qualify row_number() over (partition by loanAccountNumber order by deftpd30_outstanding_principal desc )=1),
default_outstanding_principal as 
(select digitalLoanAccountId,allaccount.loanAccountNumber,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,
allaccount.firstDueDate,allaccount.secondDueDate,allaccount.thirdDueDate,defFPD30 loan_defFPD30,defSPD30 loan_defSPD30,defTPD30 loan_defTPD30  from all_default_account allaccount
left join defpd10_pricipal_outstanding day10dpd on allaccount.loanAccountNumber=day10dpd.loanAccountNumber
left join defpd30_pricipal_outstanding firstdpd on allaccount.loanAccountNumber=firstdpd.loanAccountNumber
left join defspd30_pricipal_outstanding seconddpd on allaccount.loanAccountNumber=seconddpd.loanAccountNumber
left join deftpd30_pricipal_outstanding thirddpd on allaccount.loanAccountNumber=thirddpd.loanAccountNumber)
select * from default_outstanding_principal;
"""
query_job = client.query(query)
# Wait for the job to complete.
query_job.result()
print(f"Created table: {query_job.destination}")

Created table: prj-prod-dataplatform.dap_ds_poweruser_playground.lat_default_outstanding_principal
